In [11]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

import gzip
from pyfaidx import Fasta
from collections import Counter

In [12]:
def read_fasta_gz_to_dict(fasta_gz_path):
    """
    从 .fasta.gz 文件读取并转换为 {id: sequence} 字典。
    """
    seq_dict = {}
    with gzip.open(fasta_gz_path, "rt") as f:
        seq_id = None
        seq_lines = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if seq_id is not None:
                    seq_dict[seq_id] = "".join(seq_lines)
                seq_id = line[1:]  # 去掉 '>'
                seq_lines = []
            else:
                seq_lines.append(line)
        # 处理最后一条序列
        if seq_id is not None:
            seq_dict[seq_id] = "".join(seq_lines)
    return seq_dict

In [13]:
dic = read_fasta_gz_to_dict('../data/Tewhey_Lab_MPRA/Jurkat/ENCFF701QCG.fasta.gz')
dic

{'5860set_5overlaps_oligo100': 'CTGTCTGAACAATGTTAGTTTTCAAACATCCAAGATTCCCTCCTCTGTGGCCAGGCATCTAGTGTCAGCTAGCACTCTCTTTTGTATCAATCCTCTATTGGATAGGGGCAGAGCTAGGCTATTTACCTTTCCCCCTGGAGAGGTGGGTACCTTATCTATTGATTCTGGGTATTGGGGGTGGGATATATGGATCTAAAGAA',
 '5860set_5overlaps_oligo1000': 'TAGAACAGTGCCTGACTCAAGCACTCAGATATTCAATCAATGAACAATGAATGTATAAATCAATCACCTATGTGCCAGACACTTTGCTCCTTACAGAGCATTATGGATTTAATTGTGTCCCACCCCTAAAATTTATATGTTGAAGTACTAAGCCCCAGCACCTCAGAATGTGACCTTATTTGGAAATAAGGCTGTTGAAG',
 '5860set_5overlaps_oligo1001': 'CACACGAAGAATGCCTTGTGAAGACTTGGAGTCATGCTGCCACAAGCCAAAGAACTATCAGAAGTTAGGAGAAAGGCCAGGTACGGTGGTTCACACCTGTAATCCCAAACTTTGGGAGGTCGAGGCAGGTGGATTGCATGAGGTCAGGAGTTCGAGACCAGCCTGGCCAACATGATGAAACCCTGTCTCTACCAAAAGTA',
 '5860set_5overlaps_oligo1002': 'CACACGAAGAATGCCTTGTGAAGACTTGGAGTCATGCTGCCACAAGCCAAAGAACTATCAGAAGTTAGGAGAAAGGCCAGGTACGGTGGTTCACACCTGCAATCCCAAACTTTGGGAGGTCGAGGCAGGTGGATTGCATGAGGTCAGGAGTTCGAGACCAGCCTGGCCAACATGATGAAACCCTGTCTCTACCAAAAGTA',
 '5860set_5overlaps_oligo1003': 'AGTCATCCAGTTTGTGGTACTTTA

In [16]:
df = pd.read_csv('../data/Tewhey_Lab_MPRA/Jurkat/ENCFF511FCA.tsv', sep='\t')
df

,ID,SNP,chr,pos,ref_allele,alt_allele,project,window,strand,allele,haplotype,bash,ctrl_exp,DNA_mean,ctrl_mean,exp_mean,log2FoldChange,lfcSE,stat,pvalue,padj
0,5860set_5overlaps_oligo100,5860set_5overlaps_oligo100,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,1187.500,414.040,496.332,0.262,0.344,0.761,0.447,0.851
1,5860set_5overlaps_oligo1000,5860set_5overlaps_oligo1000,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,900.250,313.277,503.648,0.684,0.252,2.715,0.007,0.051
2,5860set_5overlaps_oligo1001,5860set_5overlaps_oligo1001,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,1173.000,408.567,362.595,-0.173,0.211,-0.817,0.414,0.833
3,5860set_5overlaps_oligo1002,5860set_5overlaps_oligo1001,NaN,NaN,NaN,NaN,5860set,center,fwd,alt,ref,NaN,5860set,748.500,260.931,285.788,0.132,0.473,0.280,0.779,0.968
4,5860set_5overlaps_oligo1003,5860set_5overlaps_oligo1003,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,283.500,99.628,88.939,-0.165,0.718,-0.229,0.819,0.975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59875,YRI_rs9988792_ref,rs9988792,NaN,NaN,NaN,NaN,CMS,center,fwd,ref,ref,NaN,CMS,949.000,330.871,296.788,-0.156,0.393,-0.397,0.691,0.948
59876,YRI_rs9989247_alt,rs9989247,NaN,NaN,NaN,NaN,CMS,center,fwd,alt,ref,NaN,CMS,611.250,213.367,209.375,-0.028,0.502,-0.057,0.955,0.996
59877,YRI_rs9989247_ref,rs9989247,NaN,NaN,NaN,NaN,CMS,center,fwd,ref,ref,NaN,CMS,1303.750,454.743,371.340,-0.293,0.230,-1.272,0.203,0.624
59878,YRI_rs999798_alt,rs999798,NaN,NaN,NaN,NaN,CMS,center,fwd,alt,ref,NaN,CMS,597.500,207.743,379.307,0.868,0.218,3.975,0.000,0.001


In [18]:
df['seq'] = df['ID'].map(dic)

In [19]:
df

,ID,SNP,chr,pos,ref_allele,alt_allele,project,window,strand,allele,haplotype,bash,ctrl_exp,DNA_mean,ctrl_mean,exp_mean,log2FoldChange,lfcSE,stat,pvalue,padj,seq
0,5860set_5overlaps_oligo100,5860set_5overlaps_oligo100,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,1187.500,414.040,496.332,0.262,0.344,0.761,0.447,0.851,CTGTCTGAACAATGTTAGTTTTCAAACATCCAAGATTCCCTCCTCT...
1,5860set_5overlaps_oligo1000,5860set_5overlaps_oligo1000,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,900.250,313.277,503.648,0.684,0.252,2.715,0.007,0.051,TAGAACAGTGCCTGACTCAAGCACTCAGATATTCAATCAATGAACA...
2,5860set_5overlaps_oligo1001,5860set_5overlaps_oligo1001,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,1173.000,408.567,362.595,-0.173,0.211,-0.817,0.414,0.833,CACACGAAGAATGCCTTGTGAAGACTTGGAGTCATGCTGCCACAAG...
3,5860set_5overlaps_oligo1002,5860set_5overlaps_oligo1001,NaN,NaN,NaN,NaN,5860set,center,fwd,alt,ref,NaN,5860set,748.500,260.931,285.788,0.132,0.473,0.280,0.779,0.968,CACACGAAGAATGCCTTGTGAAGACTTGGAGTCATGCTGCCACAAG...
4,5860set_5overlaps_oligo1003,5860set_5overlaps_oligo1003,NaN,NaN,NaN,NaN,5860set,center,fwd,ref,ref,NaN,5860set,283.500,99.628,88.939,-0.165,0.718,-0.229,0.819,0.975,AGTCATCCAGTTTGTGGTACTTTATTACGGCAGCTGTGGGGACTTC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59875,YRI_rs9988792_ref,rs9988792,NaN,NaN,NaN,NaN,CMS,center,fwd,ref,ref,NaN,CMS,949.000,330.871,296.788,-0.156,0.393,-0.397,0.691,0.948,TACGCAAATTTTTTTTTTTTTTTGAGATAGAGTCTTGCTCTATTGC...
59876,YRI_rs9989247_alt,rs9989247,NaN,NaN,NaN,NaN,CMS,center,fwd,alt,ref,NaN,CMS,611.250,213.367,209.375,-0.028,0.502,-0.057,0.955,0.996,GTCTTGGCTATGAGGGCTCTTTTTTGGTTCCATATGAAATTTAAAG...
59877,YRI_rs9989247_ref,rs9989247,NaN,NaN,NaN,NaN,CMS,center,fwd,ref,ref,NaN,CMS,1303.750,454.743,371.340,-0.293,0.230,-1.272,0.203,0.624,GTCTTGGCTATGAGGGCTCTTTTTTGGTTCCATATGAAATTTAAAG...
59878,YRI_rs999798_alt,rs999798,NaN,NaN,NaN,NaN,CMS,center,fwd,alt,ref,NaN,CMS,597.500,207.743,379.307,0.868,0.218,3.975,0.000,0.001,TAGCCTAGGGCGTTAAAAGTCTAAATTTTAATTCTGTCAACAACTG...


In [22]:
df['Jurkat'] = df['log2FoldChange']


df = df[['ID', 'Jurkat', 'seq']]

In [23]:
df.to_csv('../data/Tewhey_Lab_MPRA/Jurkat.csv', index=False)

In [5]:
# cell_type = 'GM12878'
# cell_type = 'A549'
# cell_type = 'HCT116'
cell_type = 'Jurkat'

In [6]:
df_list = []

for file_name in os.listdir(f'{root_dir}/{cell_type}'):
    if file_name.endswith('.tsv'):
        df = pd.read_csv(f'{root_dir}/{cell_type}/{file_name}', sep='\t', low_memory=False)



        c = Counter(df['ctrl_exp'])
        print(c.most_common())
        df['experiment'] = c.most_common()[0][0]
        ctrl_pos_mean = df[df['ctrl_exp'] == 'ctrl_pos']['log2FoldChange'].mean()
        ctrl_neg_mean = df[df['ctrl_exp'] == 'ctrl_neg']['log2FoldChange'].mean()
        total_mean = df['log2FoldChange'].mean()
        print(ctrl_pos_mean, ctrl_neg_mean, total_mean)
        df['log2FoldChangeOverControl'] = (df['log2FoldChange'] - ctrl_neg_mean) / (ctrl_pos_mean - ctrl_neg_mean)
        df_list.append(df)
df = pd.concat(df_list).reset_index(drop=True)
df

# 不能简单合并，要按照ctrl做归一化再合并
# 这里把ctrl_pos归一化到1，把ctrl_neg归一化到0，计算log2FoldChangeOverControl



df['chr'] = df['chr'].apply(lambda x: 'chr' + x)
df.rename(columns={
    'ref_allele': 'ref', 
    'alt_allele': 'alt', 
    }, inplace=True)

df = df.sort_values(by=['chr', 'pos']).reset_index(drop=True)

# 只保留ID只有5个:的
df = df[df['ID'].str.count(':') == 5].reset_index(drop=True)
# 只保留window是center的
df = df[df['window'] == 'center'].reset_index(drop=True)
# 只保留allele长度为1的或者allele为nan的
df = df[((df['ref'].str.len() == 1) & (df['alt'].str.len() == 1)) | (df['ref'].isna())].reset_index(drop=True)
df




# 合并相同ID的序列，活性使用平均值
# df = df.groupby(['ID'], as_index=False).agg(lambda x: x.mean() if x.dtype == float else x.iloc[0]).reset_index(drop=True)
df = df.groupby('ID', as_index=False).agg(
    {col: 'mean' if df[col].dtype == 'float' else 'first' for col in df.columns}
)
# 删除重复'chr', 'pos', 'allele'的序列，注意这些序列的ID不同，所以是同一个位点，做了两次突变，生成了4条序列的情况，直接删除后两条
df = df.drop_duplicates(subset=['chr', 'pos', 'allele'])
df = df.sort_values(by=['chr', 'pos', 'allele']).reset_index(drop=True)
df





genome = Fasta('/home/hxcai/genome/hg19.fa')

def process_row(row):
    chr, pos, ref, alt, allele = row['chr'], row['pos'], row['ref'], row['alt'], row['allele']

    genome_ref_base = genome[chr][pos-1].seq.upper()
    correct_flag = genome_ref_base == ref  # 位置正确的标志

    seq = genome[chr][pos-100:pos+100].seq.upper()
    if allele != ref:
        seq = seq[:99] + alt + seq[100:]

    return seq, correct_flag

df[['seq', 'flags']] = df.apply(process_row, axis=1, result_type='expand')
print(len(df), df['flags'].sum())



df = df[['ID', 'chr', 'pos', 'allele', 'seq', 'log2FoldChange', 'log2FoldChangeOverControl']]
df.to_csv(f'TewheyMPRA_{cell_type}.csv', index=False)

KeyError: 'ctrl_exp'